# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.utils import shuffle
from sklearn.svm import SVR

# Preprocessing

In [ ]:
END_TRAIN_DATE = 2016
VAL_DATE = 2017
TEST_DATE = 2018
TARGET = "TOTAL_CONS"

In [ ]:
df = pd.read_csv("/content/FINAL_DATASET.csv")
df.set_index(pd.to_datetime(df["Timestamp"]), inplace=True)
df.drop("Timestamp", axis=1, inplace=True)
df

,TOTAL_CONS,Weekend,Holiday,temp,humidity,cos_hour,sin_hour,cos_weekday,sin_weekday
Timestamp,,,,,,,,,
2010-10-01 00:00:00,4390.054,0,0,19.70,60.0,1.000000,0.000000,0.841129,0.540834
2010-10-01 01:00:00,4046.071,0,0,19.61,60.0,0.999132,0.041655,0.841129,0.540834
2010-10-01 02:00:00,3885.451,0,0,18.37,72.0,0.996530,0.083237,0.841129,0.540834
2010-10-01 03:00:00,3808.100,0,0,18.05,72.0,0.992198,0.124675,0.841129,0.540834
2010-10-01 04:00:00,3782.623,0,0,17.92,72.0,0.986143,0.165896,0.841129,0.540834
...,...,...,...,...,...,...,...,...,...
2020-11-22 19:00:00,4281.942,1,0,11.83,62.0,0.702660,0.711525,0.654600,0.755975
2020-11-22 20:00:00,4091.488,1,0,11.76,66.0,0.672412,0.740177,0.654600,0.755975
2020-11-22 21:00:00,3738.827,1,0,11.61,71.0,0.640997,0.767544,0.654600,0.755975


In [ ]:
TARGET_POS = np.where(df.columns == TARGET)

In [ ]:
train_val_df = df[df.index.year <= VAL_DATE]
test_df = df[(df.index.year == TEST_DATE)]

In [ ]:
scaler = MinMaxScaler()
train_val_scaled = scaler.fit_transform(train_val_df)
train_val_df_scaled = pd.DataFrame(train_val_scaled,
                                   columns=train_val_df.columns,
                                   index=train_val_df.index)
test_scaled = scaler.transform(test_df)
test_df_scaled = pd.DataFrame(test_scaled,
                              columns=test_df.columns,
                              index=test_df.index)

In [ ]:
train_val_df_scaled = shuffle(train_val_df_scaled)
test_df_scaled = shuffle(test_df_scaled)

In [ ]:
train_val_df_scaled

,TOTAL_CONS,Weekend,Holiday,temp,humidity,cos_hour,sin_hour,cos_weekday,sin_weekday
Timestamp,,,,,,,,,
2017-01-05 02:00:00,0.233692,0.0,0.0,0.292033,0.826087,0.991837,0.101727,0.738160,0.549716
2012-05-18 16:00:00,0.286244,0.0,0.0,0.395687,0.804348,0.496342,0.755737,0.540038,0.715412
2016-02-15 16:00:00,0.300604,0.0,0.0,0.560739,0.271739,0.496342,0.755737,1.000000,0.000000
2016-01-24 00:00:00,0.366795,1.0,0.0,0.140845,0.663043,1.000000,0.000000,0.000000,1.000000
2014-10-28 09:00:00,0.347599,0.0,1.0,0.380282,0.684783,0.836533,0.447638,0.970507,0.188329
...,...,...,...,...,...,...,...,...,...
2014-06-05 01:00:00,0.164616,0.0,0.0,0.477333,0.869565,0.997958,0.050908,0.738160,0.549716
2013-01-28 18:00:00,0.653015,0.0,0.0,0.313160,0.576087,0.368851,0.833060,1.000000,0.000000
2014-05-05 09:00:00,0.314195,0.0,0.0,0.488116,0.652174,0.836533,0.447638,1.000000,0.000000


# ML-MODELS: NO SCADA

In [ ]:
X_train_val_df, y_train_val_df = train_val_df_scaled.drop(TARGET, axis=1), train_val_df_scaled[TARGET]
X_test_df, y_test_df = test_df_scaled.drop(TARGET, axis=1), test_df_scaled[TARGET]

In [ ]:
# XGBoost
xgb = XGBRegressor()
xgb.fit(X_train_val_df, y_train_val_df)
y_preds_scaled = xgb.predict(X_test_df)

XGBoost: MAPE = 14.398975146595257 %


In [ ]:
# SVR 
svr = SVR()
svr.fit(X_train_val_df, y_train_val_df)
y_preds_scaled = svr.predict(X_test_df)

In [ ]:
y_preds = scaler.data_min_[TARGET_POS] + y_preds_scaled * (scaler.data_max_[TARGET_POS] - scaler.data_min_[TARGET_POS])
y_test = scaler.data_min_[TARGET_POS] + y_test_df.to_numpy() * (scaler.data_max_[TARGET_POS] - scaler.data_min_[TARGET_POS])

xgb_results = pd.DataFrame(
    {
      "real": y_test,
      "predictions": y_preds
    },
    index=y_test_df.index
)

xgb_results['abs_error'] = np.abs(xgb_results["real"] - xgb_results["predictions"])
xgb_results['ape'] = np.where(xgb_results["real"] == 0, np.NaN, 100 * xgb_results['abs_error'] / xgb_results["real"])
mape = xgb_results['ape'].mean()

print(f"MAPE = {mape} %")

XGBoost: MAPE = 15.297341204316863 %
